# RAG Chatbot - HuggingFace Version (Hong Kong Compatible)

## What This Does

This notebook creates a chatbot that answers questions about your PDF documents using **HuggingFace Inference API** (works in Hong Kong without VPN).

**You Need:**
- PDF files in your Google Drive
- A free HuggingFace API token (~300 requests/hour)
- Internet connection

**Cost:** 100% FREE

**Why HuggingFace?** Google Gemini requires VPN in Hong Kong/China. HuggingFace works without VPN and offers access to many open-source models.

---

**Need help?** See STUDENT_GUIDE.md for detailed instructions.

---
## Step 1: Install Libraries

This installs the required tools. Takes about 30 seconds.

In [ ]:
# Install all required packages
!pip install -q chromadb gradio pypdf sentence-transformers huggingface_hub vaderSentiment

print("✅ All libraries installed successfully!")
print("\nℹ️  Note: You may see dependency warnings about 'opentelemetry' packages.")
print("   These are non-critical and won't affect functionality. You can safely ignore them.")

---
## Step 2: Load Libraries

Load the tools we just installed.

In [ ]:
import os
import time
import asyncio
import gradio as gr
from huggingface_hub import InferenceClient
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from google.colab import drive
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

---
## Step 3: Connect Google Drive

**Steps:**
1. Click the link that appears
2. Choose your Google account
3. Click "Allow"

Your files will be at: `/content/drive/MyDrive/`

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")
print("📁 Your files are available at: /content/drive/MyDrive/")

---
## Step 4: Configuration - CHANGE THESE! ✏️

**⚠️ IMPORTANT: You must edit the values below**

### Get Your FREE HuggingFace Token:
1. Go to: https://huggingface.co/
2. Sign up for free account (email + password)
3. Go to: https://huggingface.co/settings/tokens
4. Click "New token"
5. Name it: "Empathy Chatbot"
6. Type: "Read"
7. Click "Generate"
8. Copy the token (starts with `hf_`)
9. Paste it in the next cell where it says `YOUR_API_TOKEN_HERE`

**Free Tier:** ~300 requests per hour (plenty for this course!)

### Add Your PDF Files:
- Upload PDFs to your Google Drive
- Update the `PDF_PATHS` list with your file paths

In [ ]:
# ============================================================================
# 1. API TOKEN - ADD YOUR TOKEN HERE! ✏️
# ============================================================================
# Get your FREE token from: https://huggingface.co/settings/tokens
# Create a "Fine-grained" token with "Make calls to Inference Providers" enabled

HUGGINGFACE_TOKEN = "YOUR_HUGGINGFACE_TOKEN_HERE"  # ✏️ REPLACE THIS with your token (starts with hf_)

# ============================================================================
# 2. MODEL SELECTION - RECOMMENDED: Meta-Llama-3.1-8B ✏️
# ============================================================================

MODEL_NAME = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # ✅ Works with chat_completion, high quality

# Alternative models (uncomment to try):
# MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"  # Fastest, most reliable
# MODEL_NAME = "google/gemma-2-2b-it"  # Lightweight, instruction-tuned
# MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"  # Good quality (use text_generation instead)

# ============================================================================
# 3. PERSONA - CUSTOMIZE THIS! ✏️
# ============================================================================

PERSONA_NAME = "Your Persona Name"  # ✏️ CHANGE THIS - e.g., "Albert Einstein", "Oprah Winfrey"

# ✏️ CUSTOMIZE THIS: Describe your persona's speaking style and personality
PERSONA_DESCRIPTION = """
Replace this entire section with your persona's description.

Template:
You are [NAME], [brief description/title/role].
You speak in a [adjective] manner, using [characteristic words/phrases].
You emphasize [what they care about] and often [communication patterns].

Instructions for customization:
- Describe HOW they speak (tone, word choice, sentence structure)
- Include specific phrases or words they commonly use
- Mention what topics/themes they emphasize
- Note any unique speaking patterns or habits
- Keep it focused on communication style, not just biographical facts

Example:
"You are Marie Curie, pioneering scientist. You speak precisely and scientifically,
using terms like 'research,' 'experiment,' and 'discovery.' You emphasize evidence-based
reasoning and the importance of persistence in scientific work."
"""

# Tip: Describe how your person speaks and thinks

# ============================================================================
# 4. RESPONSE SETTINGS - OPTIONAL
# ============================================================================

TEMPERATURE = 0.7  # Creativity level (0.0 = focused, 1.0 = creative)

MAX_OUTPUT_TOKENS = 300  # Maximum response length

NUM_RETRIEVED_DOCS = 7  # How many document pieces to search

# ============================================================================
# 5. CONVERSATION MEMORY - OPTIONAL
# ============================================================================

CONVERSATION_MEMORY = 3  # How many previous message pairs to remember (0 = no memory)
# Higher values = more context but slower responses and higher token usage
# Recommended: 3-5 for natural conversation

# 💡 EXAMPLES:
#
# CONVERSATION_MEMORY = 0  # No memory - each question treated independently
#   ✅ Best for: Quick Q&A, unrelated questions
#   ✅ Pros: Fast, low token usage
#   ❌ Cons: Can't reference previous messages
#
# CONVERSATION_MEMORY = 3  # Remember last 3 exchanges (RECOMMENDED)
#   ✅ Best for: Natural conversation, follow-up questions
#   ✅ Pros: Contextual responses, feels more human
#   ❌ Cons: Slightly slower
#
# CONVERSATION_MEMORY = 10  # Remember last 10 exchanges
#   ✅ Best for: Long, complex discussions
#   ✅ Pros: Deep context, can reference far back
#   ❌ Cons: Slower, higher token usage, may hit limits

# ============================================================================
# 5B. DEBUG MODE - TESTING TOOL 🧪
# ============================================================================

DEBUG_MEMORY = False  # Set to True to see conversation history sent to API
# When enabled, you'll see:
#   - How many messages are sent to the API
#   - The exact content of each message (system/user/assistant)
#   - Whether conversation memory is working correctly
# Useful for testing and understanding how memory works!

# ============================================================================
# 6. SOURCE CITATIONS - OPTIONAL
# ============================================================================

SHOW_SOURCES = True  # True = show which PDFs were used, False = hide

# ============================================================================
# 7. CHUNKING SETTINGS - OPTIONAL
# ============================================================================
# How to split your PDFs into searchable pieces

CHUNK_SIZE = 1000  # Characters per chunk (500-2000 recommended)
OVERLAP = 200      # Overlap between chunks (keeps context)

# 💡 EXAMPLES - When to adjust:
#
# Example 1: SHORT & PRECISE (for quick facts)
#   CHUNK_SIZE = 500
#   OVERLAP = 100
#   ✅ Best for: Short Q&A, specific facts, definitions
#   ✅ Pros: Fast, precise answers
#   ❌ Cons: May miss broader context
#
# Example 2: LONG & CONTEXTUAL (for complex topics)
#   CHUNK_SIZE = 1500
#   OVERLAP = 300
#   ✅ Best for: Detailed explanations, complex reasoning
#   ✅ Pros: Rich context, complete thoughts
#   ❌ Cons: Slower, may include irrelevant info
#
# 🎯 CURRENT (BALANCED): 1000 chars, 200 overlap
#   ✅ Works well for general conversation and empathy training

# ============================================================================
# 8. PDF FILES - ADD YOUR PDFS HERE! ✏️
# ============================================================================
# Format: "/content/drive/MyDrive/folder_name/file_name.pdf"
# Example: "/content/drive/MyDrive/School/Materials/speech.pdf"

PDF_PATHS = [
    "/content/drive/MyDrive/your_folder/document1.pdf",  # ✏️ REPLACE with your PDF path
    "/content/drive/MyDrive/your_folder/document2.pdf",  # ✏️ Add more PDFs as needed
    "/content/drive/MyDrive/your_folder/document3.pdf",
    # Add more PDFs here...
]

# 💡 TIP: To get the exact path:
#   1. Click folder icon on left sidebar in Colab
#   2. Navigate to your PDF file
#   3. Right-click → "Copy path"
#   4. Paste here

# ============================================================================
# SETUP (Don't change this part)
# ============================================================================
client = InferenceClient(token=HUGGINGFACE_TOKEN)

print("✅ Configuration complete!")
print(f"📋 Persona: {PERSONA_NAME}")
print(f"🤖 Model: {MODEL_NAME}")
print(f"🌏 API: HuggingFace (Hong Kong compatible)")
print(f"📄 PDF files: {len(PDF_PATHS)}")
print(f"🌡️  Creativity: {TEMPERATURE}")
print(f"📊 Search pieces: {NUM_RETRIEVED_DOCS}")
print(f"🧠 Conversation memory: {CONVERSATION_MEMORY} message pairs")
print(f"🧪 Debug mode: {'ON 🔍' if DEBUG_MEMORY else 'OFF'}")
print(f"📚 Show sources: {'ON ✅' if SHOW_SOURCES else 'OFF'}")
print(f"📏 Chunk size: {CHUNK_SIZE} chars (overlap: {OVERLAP})")

---
## 🧪 Step 5: Test API Connection

**Run this first!** This checks if your API token works.

✅ If successful: Continue to next step  
❌ If failed: Check your API token and try again

In [ ]:
print("🧪 Testing HuggingFace API connection...")
print("=" * 60)

try:
    # Test using chat_completion API (better for conversational AI)
    test_response = client.chat_completion(
        messages=[
            {"role": "system", "content": PERSONA_DESCRIPTION},
            {"role": "user", "content": "Say 'Hello! API is working!' in a friendly, enthusiastic style."}
        ],
        model=MODEL_NAME,
        max_tokens=50,
        temperature=0.7
    )
    
    # Extract response from chat completion format
    response_text = test_response.choices[0].message.content
    
    print("✅ SUCCESS! HuggingFace API is working!")
    print(f"\nTest Response: {response_text}")
    print("\n" + "=" * 60)
    print("✅ You can proceed with the rest of the notebook!")
    
except Exception as e:
    error_str = str(e).lower()
    print(f"❌ API TEST FAILED!")
    print(f"Error: {str(e)}")
    print("\n" + "=" * 60)
    print("⚠️  STOP! Fix this issue before proceeding:")
    
    if "503" in str(e) or "loading" in error_str:
        print("  🔄 Model is loading for the first time (can take 20-30 seconds)")
        print("  💡 SOLUTION: Wait 30 seconds and run this cell again")
    elif "invalid" in error_str or "token" in error_str or "401" in str(e) or "403" in str(e):
        print("  🔑 Token issue detected")
        print("  💡 SOLUTION:")
        print("     1. Go to https://huggingface.co/settings/tokens")
        print("     2. Create 'Fine-grained' token")
        print("     3. Enable 'Make calls to Inference Providers' under Inference section")
        print("     4. Copy new token to Cell 8")
    elif "model" in error_str and ("not found" in error_str or "does not exist" in error_str):
        print("  🤖 Model not available")
        print("  💡 SOLUTION: Try 'microsoft/Phi-3.5-mini-instruct' in Cell 8")
    else:
        print("  1. Check your API token is correct (starts with hf_)")
        print("  2. Check your internet connection")
        print("  3. Visit https://huggingface.co/settings/tokens to verify your token")
        print("  4. Make sure token has 'Inference' permissions enabled")

---
## 🧪 Step 5B: Test Conversation Memory (Optional)

**What this does:** Tests if conversation memory is working correctly.

This cell simulates a multi-turn conversation and shows you:
- How many messages are sent to the API
- The exact format of the conversation history
- Whether the bot can reference previous exchanges

**When to run this:**
- First time using the notebook (verify memory works)
- After changing `CONVERSATION_MEMORY` setting
- If responses don't seem to remember context

**Instructions:**
1. Make sure Cell 10 (API test) passed ✅
2. Run this cell
3. Check the output to see how memory is structured

In [ ]:
print("🧪 TESTING CONVERSATION MEMORY")
print("=" * 80)
print(f"\nConfiguration:")
print(f"   • CONVERSATION_MEMORY = {CONVERSATION_MEMORY}")
print(f"   • Model: {MODEL_NAME}")
print(f"   • API: HuggingFace")

# Simulate a conversation history (what Gradio would provide)
print(f"\n📝 Simulating a 3-turn conversation...\n")

simulated_history = [
    ["What are your main beliefs?", "I believe in scientific progress and evidence."],
    ["Tell me more", "Science requires rigorous experimentation and peer review."],
    ["Why is that important?", "It prevents bias and ensures reproducible results."]
]

print("Conversation so far:")
for i, (user, bot) in enumerate(simulated_history, 1):
    print(f"   {i}. User: {user[:50]}...")
    print(f"      Bot:  {bot[:50]}...")

# Show what would be sent to the API
print(f"\n🔍 Building message array for API...")
print(f"   Memory setting: {CONVERSATION_MEMORY} exchanges")

# Simulate the message building logic from Cell 16
PERSONA_TEST = "You are a helpful assistant."
context_test = "Test context from documents."

messages = [
    {
        "role": "system",
        "content": f"{PERSONA_TEST}\n\nContext: {context_test}"
    }
]

# Add conversation history (limited by CONVERSATION_MEMORY)
if CONVERSATION_MEMORY > 0:
    recent_history = simulated_history[-(CONVERSATION_MEMORY):]
    print(f"   Including last {len(recent_history)} exchanges from history")
    
    for user_msg, bot_msg in recent_history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})
else:
    print(f"   ⚠️  Memory DISABLED - no history included")

# Add current question
current_question = "What else can you tell me?"
messages.append({"role": "user", "content": current_question})

# Display message structure
print(f"\n📊 Message array to be sent to API:")
print(f"   Total messages: {len(messages)}")
print(f"   Breakdown:")
print(f"      - System message: 1")
print(f"      - History pairs: {(len(messages) - 2) // 2}")
print(f"      - Current question: 1")

print(f"\n📋 Full message structure:")
print("-" * 80)
for i, msg in enumerate(messages):
    role_emoji = "🖥️" if msg['role'] == "system" else ("👤" if msg['role'] == "user" else "🤖")
    content_preview = msg['content'][:60].replace('\n', ' ')
    print(f"   [{i}] {role_emoji} {msg['role']:10} | {content_preview}...")
print("-" * 80)

# Verify memory is working as expected
expected_messages = 1 + (CONVERSATION_MEMORY * 2 if CONVERSATION_MEMORY > 0 else 0) + 1
actual_messages = len(messages)

print(f"\n✅ VERIFICATION:")
print(f"   Expected: {expected_messages} messages (1 system + {CONVERSATION_MEMORY*2 if CONVERSATION_MEMORY > 0 else 0} history + 1 current)")
print(f"   Actual:   {actual_messages} messages")

if actual_messages == expected_messages:
    print(f"   ✅ PASS - Conversation memory structure is correct!")
    if CONVERSATION_MEMORY > 0:
        print(f"   ✅ Memory is ENABLED - bot will remember last {CONVERSATION_MEMORY} exchanges")
        print(f"   ✅ Follow-up questions will work naturally")
    else:
        print(f"   ⚠️  Memory is DISABLED - each question treated independently")
else:
    print(f"   ❌ FAIL - Expected {expected_messages} but got {actual_messages}")
    print(f"   💡 Check CONVERSATION_MEMORY setting in Cell 8")

print(f"\n{'='*80}")
print("💡 To test with real API:")
print("   1. Continue to Cell 12 (process PDFs)")
print("   2. Run all cells through Cell 19 (launch chat)")
print("   3. Set DEBUG_MEMORY = True in Cell 8 to see this output during chat")
print("   4. Try asking follow-up questions like 'tell me more' or 'what else?'")
print(f"{'='*80}\n")

---
## Step 6: Read PDF Files

This reads your PDFs and splits them into small pieces for searching.

**Time:** 1-2 minutes depending on file size

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Get text from a PDF file."""
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"❌ Error reading {pdf_path}: {str(e)}")
        return ""

def chunk_text(text, chunk_size=CHUNK_SIZE, overlap=OVERLAP):
    """Split text into small pieces (chunks) for better searching.
    
    Uses settings from Cell 8:
    - chunk_size: Characters per chunk
    - overlap: Characters that overlap between chunks (prevents splitting mid-sentence)
    """
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        
        if chunk.strip():
            chunks.append(chunk)
        
        start += chunk_size - overlap  # Move forward, keep overlap
    
    return chunks

# Process all PDFs
print("📖 Reading PDF files...\n")
all_chunks = []
metadata = []

for idx, pdf_path in enumerate(PDF_PATHS):
    print(f"Processing: {pdf_path}")
    
    if not os.path.exists(pdf_path):
        print(f"⚠️  File not found - {pdf_path}")
        continue
    
    text = extract_text_from_pdf(pdf_path)
    
    if text:
        chunks = chunk_text(text)  # Split into small pieces
        all_chunks.extend(chunks)
        
        # Save info about where each chunk came from
        for chunk_idx, chunk in enumerate(chunks):
            metadata.append({
                "source": os.path.basename(pdf_path),
                "chunk_id": chunk_idx,
                "total_chunks": len(chunks)
            })
        
        print(f"  ✅ Created {len(chunks)} pieces")
    else:
        print(f"  ⚠️  No text found")

print(f"\n✅ Done!")
print(f"📊 Total pieces: {len(all_chunks)}")
print(f"📏 Using chunk size: {CHUNK_SIZE} chars with {OVERLAP} char overlap")

if len(all_chunks) == 0:
    print("\n⚠️  WARNING: No text found in PDFs!")
    print("Check: File paths correct? PDFs not password-protected?")

---
## Step 7: Create Search Database

This creates a searchable database from your PDFs.

**Time:** 1-2 minutes

In [ ]:
# Store which PDFs were used for the last answer
last_sources_used = []

def retrieve_relevant_context(query, n_results=NUM_RETRIEVED_DOCS):
    """Find relevant pieces from your PDFs based on the question."""
    global last_sources_used
    try:
        # Convert question to searchable numbers
        query_embedding = embedding_model.encode([query])
        
        # Search database for matching pieces
        results = collection.query(
            query_embeddings=query_embedding.tolist(),
            n_results=min(n_results, collection.count())
        )
        
        documents = results['documents'][0] if results['documents'] else []
        metadatas = results['metadatas'][0] if results['metadatas'] else []
        
        # Track which PDFs were used
        last_sources_used = []
        if metadatas:
            seen_sources = set()
            for meta in metadatas[:3]:  # Only track the 3 pieces we actually use
                source_name = meta.get('source', 'Unknown')
                if source_name not in seen_sources:
                    last_sources_used.append(source_name)
                    seen_sources.add(source_name)
        
        return documents
    except Exception as e:
        print(f"Error searching: {str(e)}")
        last_sources_used = []
        return []

def generate_response_sync(question, chat_history=None):
    """Get answer from AI using relevant PDF pieces + conversation memory."""
    # Find relevant pieces from PDFs
    context_docs = retrieve_relevant_context(question)
    
    # 🎯 TEACHING NOTE: Context Size Settings
    # We use TOP 3 CHUNKS (out of 7 searched) for better fact-grounding
    # More chunks = more facts, but can confuse the model if too many
    # 2000 chars ≈ 500 tokens ≈ 1-2 paragraphs of context
    if context_docs:
        context_docs = context_docs[:3]  # 📝 Use top 3 most relevant chunks
        context = "\n\n".join(context_docs)
        context = context[:2000]  # 📝 Limit to 2000 characters maximum
    else:
        context = "No relevant documents found."
    
    # 🎯 TEACHING NOTE: Conversation Memory Implementation
    # We build a message history to give the AI context of previous exchanges
    # This allows natural follow-up questions like "tell me more" or "what about X?"
    # Memory is limited by CONVERSATION_MEMORY setting to balance context vs speed
    
    # Start with system message (persona + grounding instructions + context)
    messages = [
        {
            "role": "system",
            "content": f"""{PERSONA_DESCRIPTION}

IMPORTANT: Answer using ONLY the specific facts and information from the context below.
If the context doesn't contain the answer, say "I don't have that information in my documents."

Context from documents:
{context}

Instructions:
- Use actual facts, quotes, and details from the context
- Cite specific information mentioned in the documents
- Keep your answer brief and focused (around 50 words)
- Don't add information not in the context"""
        }
    ]
    
    # 🎯 TEACHING NOTE: Adding Conversation History
    # Gradio's chat_history format: [[user_msg1, bot_msg1], [user_msg2, bot_msg2], ...]
    # We convert this to the API's message format: [{role: user, content: ...}, {role: assistant, content: ...}]
    # Only include the last N exchanges (controlled by CONVERSATION_MEMORY)
    
    if chat_history and CONVERSATION_MEMORY > 0:
        # Get only the most recent exchanges
        recent_history = chat_history[-(CONVERSATION_MEMORY):]
        
        # Add each exchange to messages
        for user_msg, bot_msg in recent_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": bot_msg})
    
    # Add current question
    messages.append({"role": "user", "content": question})
    
    # 🎯 DEBUG OUTPUT - Show conversation memory structure
    if DEBUG_MEMORY:
        print("\n" + "="*80)
        print("🧠 DEBUG: CONVERSATION MEMORY")
        print("="*80)
        print(f"📊 Sending {len(messages)} messages to HuggingFace API:")
        print(f"   • System message: 1")
        history_count = (len(messages) - 2) // 2 if len(messages) > 2 else 0
        print(f"   • History pairs: {history_count} (from last {CONVERSATION_MEMORY} exchanges)")
        print(f"   • Current question: 1")
        print(f"\n📋 Message structure:")
        for i, msg in enumerate(messages):
            role_emoji = "🖥️" if msg['role'] == "system" else ("👤" if msg['role'] == "user" else "🤖")
            content_preview = msg['content'][:60].replace('\n', ' ').strip()
            print(f"   [{i}] {role_emoji} {msg['role']:10} | {content_preview}...")
        print("="*80 + "\n")
    
    try:
        # 🎯 TEACHING NOTE: API Call Parameters
        # - max_tokens: Controls output length (~75 tokens ≈ 50 words)
        # - temperature: Controls creativity (0.0=robotic, 1.0=creative)
        #   Lower temperature = more factual, sticks to documents
        # - model: Which AI model to use (Meta-Llama-3.1-8B-Instruct)
        # - messages: Full conversation history (system + previous + current)
        
        # Call HuggingFace chat_completion API
        response = client.chat_completion(
            messages=messages,
            model=MODEL_NAME,
            max_tokens=75,  # 📝 ~50 words output (1 token ≈ 0.75 words)
            temperature=TEMPERATURE  # 📝 From Cell 8, controls creativity
        )
        
        # Extract text from response
        return response.choices[0].message.content
        
    except Exception as e:
        # Better error handling for HF-specific issues
        error_str = str(e).lower()
        
        if "503" in str(e) or "loading" in error_str:
            return "⏳ Model is loading... Please wait 20-30 seconds and try again."
        elif "model" in error_str and "not found" in error_str:
            return "❌ Model not available. Try using 'microsoft/Phi-3.5-mini-instruct' in Cell 8"
        elif "429" in str(e) or "rate limit" in error_str:
            return "⚠️ Rate limit reached. Wait 10-15 minutes (free tier: ~300 requests/hour)"
        else:
            # Re-raise for async wrapper to handle
            raise e

async def generate_response_async(question, chat_history=None, timeout_seconds=30):
    """Wrapper with 30-second timeout to prevent hanging."""
    try:
        # Run AI call with timeout
        response_text = await asyncio.wait_for(
            asyncio.to_thread(generate_response_sync, question, chat_history),
            timeout=timeout_seconds
        )
        return response_text
    
    except asyncio.TimeoutError:
        return "⏱️ **Timeout** - Took too long (>30 seconds). Try a simpler question."
    
    except Exception as e:
        error_str = str(e).lower()
        
        if "429" in str(e) or "quota" in error_str or "rate limit" in error_str:
            return "⚠️ **Rate Limit** - Wait 10-15 minutes and try again."
        elif "timeout" in error_str or "connection" in error_str:
            return "⚠️ **Connection Error** - Check your internet."
        elif "invalid" in error_str or "token" in error_str or "authentication" in error_str or "401" in str(e):
            return "⚠️ **API Error** - Check your HuggingFace token permissions."
        elif "503" in str(e) or "loading" in error_str:
            return "⚠️ **Model Loading** - The model is waking up. Wait 20-30 seconds and try again."
        else:
            return f"❌ **Error** - {str(e)[:100]}"

print("✅ Answer system ready!")
print("🤖 Using chat_completion API (optimized for conversation)")
print(f"🧠 Conversation memory: {'ENABLED (' + str(CONVERSATION_MEMORY) + ' exchanges)' if CONVERSATION_MEMORY > 0 else 'DISABLED'}")
print(f"🧪 Debug mode: {'ENABLED 🔍 (will show message structure in console)' if DEBUG_MEMORY else 'DISABLED (set DEBUG_MEMORY = True in Cell 8 to enable)'}")
print("📏 Context: 3 chunks, 2000 chars max")
print("📝 Output: ~50 words per response")
print("⏱️  Response time: 5-15 seconds")
if SHOW_SOURCES:
    print("📚 Source citations enabled")

---
## Step 8: Setup Question Answering

This prepares the chatbot to answer your questions.

In [ ]:
# Store which PDFs were used for the last answer
last_sources_used = []

def retrieve_relevant_context(query, n_results=NUM_RETRIEVED_DOCS):
    """Find relevant pieces from your PDFs based on the question."""
    global last_sources_used
    try:
        # Convert question to searchable numbers
        query_embedding = embedding_model.encode([query])
        
        # Search database for matching pieces
        results = collection.query(
            query_embeddings=query_embedding.tolist(),
            n_results=min(n_results, collection.count())
        )
        
        documents = results['documents'][0] if results['documents'] else []
        metadatas = results['metadatas'][0] if results['metadatas'] else []
        
        # Track which PDFs were used
        last_sources_used = []
        if metadatas:
            seen_sources = set()
            for meta in metadatas[:3]:  # Only track the 3 pieces we actually use
                source_name = meta.get('source', 'Unknown')
                if source_name not in seen_sources:
                    last_sources_used.append(source_name)
                    seen_sources.add(source_name)
        
        return documents
    except Exception as e:
        print(f"Error searching: {str(e)}")
        last_sources_used = []
        return []

def generate_response_sync(question, chat_history=None):
    """Get answer from AI using relevant PDF pieces + conversation memory."""
    # Find relevant pieces from PDFs
    context_docs = retrieve_relevant_context(question)
    
    # 🎯 TEACHING NOTE: Context Size Settings
    # We use TOP 3 CHUNKS (out of 7 searched) for better fact-grounding
    # More chunks = more facts, but can confuse the model if too many
    # 2000 chars ≈ 500 tokens ≈ 1-2 paragraphs of context
    if context_docs:
        context_docs = context_docs[:3]  # 📝 Use top 3 most relevant chunks
        context = "\n\n".join(context_docs)
        context = context[:2000]  # 📝 Limit to 2000 characters maximum
    else:
        context = "No relevant documents found."
    
    # 🎯 TEACHING NOTE: Conversation Memory Implementation
    # We build a message history to give the AI context of previous exchanges
    # This allows natural follow-up questions like "tell me more" or "what about X?"
    # Memory is limited by CONVERSATION_MEMORY setting to balance context vs speed
    
    # Start with system message (persona + grounding instructions + context)
    messages = [
        {
            "role": "system",
            "content": f"""{PERSONA_DESCRIPTION}

IMPORTANT: Answer using ONLY the specific facts and information from the context below.
If the context doesn't contain the answer, say "I don't have that information in my documents."

Context from documents:
{context}

Instructions:
- Use actual facts, quotes, and details from the context
- Cite specific information mentioned in the documents
- Keep your answer brief and focused (around 50 words)
- Don't add information not in the context"""
        }
    ]
    
    # 🎯 TEACHING NOTE: Adding Conversation History
    # Gradio's chat_history format: [[user_msg1, bot_msg1], [user_msg2, bot_msg2], ...]
    # We convert this to the API's message format: [{role: user, content: ...}, {role: assistant, content: ...}]
    # Only include the last N exchanges (controlled by CONVERSATION_MEMORY)
    
    if chat_history and CONVERSATION_MEMORY > 0:
        # Get only the most recent exchanges
        recent_history = chat_history[-(CONVERSATION_MEMORY):]
        
        # Add each exchange to messages
        for user_msg, bot_msg in recent_history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": bot_msg})
    
    # Add current question
    messages.append({"role": "user", "content": question})
    
    try:
        # 🎯 TEACHING NOTE: API Call Parameters
        # - max_tokens: Controls output length (~75 tokens ≈ 50 words)
        # - temperature: Controls creativity (0.0=robotic, 1.0=creative)
        #   Lower temperature = more factual, sticks to documents
        # - model: Which AI model to use (Meta-Llama-3.1-8B-Instruct)
        # - messages: Full conversation history (system + previous + current)
        
        # Call HuggingFace chat_completion API
        response = client.chat_completion(
            messages=messages,
            model=MODEL_NAME,
            max_tokens=75,  # 📝 ~50 words output (1 token ≈ 0.75 words)
            temperature=TEMPERATURE  # 📝 From Cell 8, controls creativity
        )
        
        # Extract text from response
        return response.choices[0].message.content
        
    except Exception as e:
        # Better error handling for HF-specific issues
        error_str = str(e).lower()
        
        if "503" in str(e) or "loading" in error_str:
            return "⏳ Model is loading... Please wait 20-30 seconds and try again."
        elif "model" in error_str and "not found" in error_str:
            return "❌ Model not available. Try using 'microsoft/Phi-3.5-mini-instruct' in Cell 8"
        elif "429" in str(e) or "rate limit" in error_str:
            return "⚠️ Rate limit reached. Wait 10-15 minutes (free tier: ~300 requests/hour)"
        else:
            # Re-raise for async wrapper to handle
            raise e

async def generate_response_async(question, chat_history=None, timeout_seconds=30):
    """Wrapper with 30-second timeout to prevent hanging."""
    try:
        # Run AI call with timeout
        response_text = await asyncio.wait_for(
            asyncio.to_thread(generate_response_sync, question, chat_history),
            timeout=timeout_seconds
        )
        return response_text
    
    except asyncio.TimeoutError:
        return "⏱️ **Timeout** - Took too long (>30 seconds). Try a simpler question."
    
    except Exception as e:
        error_str = str(e).lower()
        
        if "429" in str(e) or "quota" in error_str or "rate limit" in error_str:
            return "⚠️ **Rate Limit** - Wait 10-15 minutes and try again."
        elif "timeout" in error_str or "connection" in error_str:
            return "⚠️ **Connection Error** - Check your internet."
        elif "invalid" in error_str or "token" in error_str or "authentication" in error_str or "401" in str(e):
            return "⚠️ **API Error** - Check your HuggingFace token permissions."
        elif "503" in str(e) or "loading" in error_str:
            return "⚠️ **Model Loading** - The model is waking up. Wait 20-30 seconds and try again."
        else:
            return f"❌ **Error** - {str(e)[:100]}"

print("✅ Answer system ready!")
print("🤖 Using chat_completion API (optimized for conversation)")
print(f"🧠 Conversation memory: {'ENABLED (' + str(CONVERSATION_MEMORY) + ' exchanges)' if CONVERSATION_MEMORY > 0 else 'DISABLED'}")
print("📏 Context: 3 chunks, 2000 chars max")
print("📝 Output: ~50 words per response")
print("⏱️  Response time: 5-15 seconds")
if SHOW_SOURCES:
    print("📚 Source citations enabled")

---
## Step 8B: Initialize Empathy Analyzer

This sets up the empathy tracking system that will analyze your messages.

In [ ]:
# ============================================================================
# EMPATHY ANALYZER - Tracks 5 dimensions of empathic communication
# ============================================================================

class EmpathyAnalyzer:
    """Analyzes user messages for empathy across 5 dimensions."""
    
    def __init__(self):
        self.vader = SentimentIntensityAnalyzer()
        self.user_messages = []
        self.empathy_scores = []
        self.conversation_history = []
        
        # Empathy linguistic markers
        self.open_question_words = ['how', 'what', 'why', 'tell', 'describe', 'explain']
        self.emotion_words = [
            'feel', 'feeling', 'felt', 'emotion', 'happy', 'sad', 'angry', 
            'frustrated', 'worried', 'anxious', 'excited', 'disappointed',
            'upset', 'hurt', 'joy', 'fear', 'surprise', 'disgust', 'content',
            'grateful', 'proud', 'ashamed', 'guilty', 'nervous', 'scared'
        ]
        self.perspective_phrases = [
            'you feel', 'you might', 'from your', 'in your', 'your perspective',
            'you seem', 'you appear', 'you sound', 'for you', 'to you',
            'you think', 'you believe', 'you experience', 'your view'
        ]
        self.active_listening_phrases = [
            'tell me more', 'i understand', 'i hear', 'i see', 'help me understand',
            'that makes sense', 'i appreciate', 'thank you for sharing',
            'go on', 'continue', 'interesting', 'i get it', 'i follow'
        ]
    
    def analyze_message(self, message):
        """Analyze a single message for empathy markers."""
        message_lower = message.lower()
        
        # 1. Sentiment/Warmth (0-20 points)
        sentiment = self.vader.polarity_scores(message)
        warmth_score = max(0, min(20, (sentiment['compound'] + 1) * 10))
        
        # 2. Open Questions (0-20 points)
        open_question_count = sum(1 for word in self.open_question_words if word in message_lower)
        has_question = '?' in message
        open_score = min(20, open_question_count * 10) if has_question else 0
        
        # 3. Emotion Words (0-20 points)
        emotion_count = sum(1 for word in self.emotion_words if word in message_lower)
        emotion_score = min(20, emotion_count * 7)
        
        # 4. Perspective-Taking (0-20 points)
        perspective_count = sum(1 for phrase in self.perspective_phrases if phrase in message_lower)
        perspective_score = min(20, perspective_count * 10)
        
        # 5. Active Listening (0-20 points)
        listening_count = sum(1 for phrase in self.active_listening_phrases if phrase in message_lower)
        listening_score = min(20, listening_count * 7)
        
        # Total score
        total_score = warmth_score + open_score + emotion_score + perspective_score + listening_score
        
        return {
            'message': message,
            'warmth': warmth_score,
            'open_questions': open_score,
            'emotion_words': emotion_score,
            'perspective_taking': perspective_score,
            'active_listening': listening_score,
            'total': total_score,
            'sentiment_raw': sentiment['compound']
        }
    
    def add_user_message(self, message, bot_response):
        """Track a user message and bot response."""
        analysis = self.analyze_message(message)
        self.user_messages.append(message)
        self.empathy_scores.append(analysis)
        self.conversation_history.append({
            'user': message,
            'bot': bot_response,
            'empathy': analysis
        })
    
    def get_average_scores(self):
        """Calculate average scores across all messages."""
        if not self.empathy_scores:
            return None
        
        n = len(self.empathy_scores)
        return {
            'warmth': sum(s['warmth'] for s in self.empathy_scores) / n,
            'open_questions': sum(s['open_questions'] for s in self.empathy_scores) / n,
            'emotion_words': sum(s['emotion_words'] for s in self.empathy_scores) / n,
            'perspective_taking': sum(s['perspective_taking'] for s in self.empathy_scores) / n,
            'active_listening': sum(s['active_listening'] for s in self.empathy_scores) / n,
            'total': sum(s['total'] for s in self.empathy_scores) / n,
            'message_count': n
        }
    
    def generate_report(self):
        """Generate comprehensive empathy report."""
        if len(self.empathy_scores) < 10:
            return None
        
        avg = self.get_average_scores()
        total_score = avg['total']
        
        # Interpretation
        if total_score >= 80:
            interpretation = "Excellent - Consistently demonstrates empathic responses"
        elif total_score >= 60:
            interpretation = "Good - Regular empathic responses with room to grow"
        elif total_score >= 40:
            interpretation = "Moderate - Awareness of emotions but inconsistent"
        elif total_score >= 20:
            interpretation = "Developing - Beginning to recognize emotions"
        else:
            interpretation = "Needs Practice - Focus on foundational skills"
        
        # Recommendations
        recommendations = []
        if avg['warmth'] < 15:
            recommendations.append("Use warmer, more supportive language")
        if avg['open_questions'] < 15:
            recommendations.append("Ask more open-ended questions (what/how/why)")
        if avg['emotion_words'] < 15:
            recommendations.append("Acknowledge emotions more explicitly")
        if avg['perspective_taking'] < 15:
            recommendations.append("Practice perspective-taking phrases")
        if avg['active_listening'] < 15:
            recommendations.append("Show more active listening markers")
        
        # Format report
        report = f"""
╔═══════════════════════════════════════════════════════════╗
║           EMPATHY TRAINING ANALYSIS REPORT                ║
╚═══════════════════════════════════════════════════════════╝

📊 OVERALL EMPATHY SCORE: {total_score:.1f}/100
   {interpretation}

📈 DIMENSION BREAKDOWN:
   • Sentiment/Warmth:      {avg['warmth']:.1f}/20 {'✅' if avg['warmth'] >= 15 else '⚠️'}
   • Open Questions:        {avg['open_questions']:.1f}/20 {'✅' if avg['open_questions'] >= 15 else '⚠️'}
   • Emotion Recognition:   {avg['emotion_words']:.1f}/20 {'✅' if avg['emotion_words'] >= 15 else '⚠️'}
   • Perspective-Taking:    {avg['perspective_taking']:.1f}/20 {'✅' if avg['perspective_taking'] >= 15 else '⚠️'}
   • Active Listening:      {avg['active_listening']:.1f}/20 {'✅' if avg['active_listening'] >= 15 else '⚠️'}

📉 CONVERSATION METRICS:
   • Total Messages Analyzed: {avg['message_count']}
   • Average Sentiment: {sum(s['sentiment_raw'] for s in self.empathy_scores) / len(self.empathy_scores):.2f} (-1 to +1)
   • Questions Asked: {sum(1 for s in self.empathy_scores if s['open_questions'] > 0)}
   • Emotion Words Used: {sum(1 for s in self.empathy_scores if s['emotion_words'] > 0)} messages

💡 RECOMMENDATIONS FOR IMPROVEMENT:
"""
        if recommendations:
            for rec in recommendations:
                report += f"   • {rec}\n"
        else:
            report += "   • Great work! Keep practicing to maintain your skills\n"
        
        report += "\n✅ Report complete - Keep practicing empathic communication!"
        
        return report
    
    def export_to_csv(self):
        """Export conversation data to CSV format."""
        import csv
        from io import StringIO
        
        output = StringIO()
        writer = csv.writer(output)
        
        # Header
        writer.writerow([
            'Message #', 'User Message', 'Bot Response', 
            'Warmth', 'Open Questions', 'Emotion Words', 
            'Perspective-Taking', 'Active Listening', 'Total Score'
        ])
        
        # Data
        for i, conv in enumerate(self.conversation_history, 1):
            emp = conv['empathy']
            writer.writerow([
                i,
                conv['user'],
                conv['bot'],
                f"{emp['warmth']:.1f}",
                f"{emp['open_questions']:.1f}",
                f"{emp['emotion_words']:.1f}",
                f"{emp['perspective_taking']:.1f}",
                f"{emp['active_listening']:.1f}",
                f"{emp['total']:.1f}"
            ])
        
        return output.getvalue()

# Initialize global empathy analyzer
empathy_analyzer = EmpathyAnalyzer()

print("✅ Empathy Analyzer ready!")
print("📊 Tracking 5 dimensions:")
print("   1. Sentiment/Warmth (positive emotional tone)")
print("   2. Open Questions (exploration)")
print("   3. Emotion Recognition (naming feelings)")
print("   4. Perspective-Taking (seeing their view)")
print("   5. Active Listening (engagement)")
print("\n📝 Report will generate after 10 messages")

In [ ]:
async def chat_interface(message, history):
    """Handle chat messages with empathy tracking, conversation memory, and source citations."""
    # Get answer from AI (with conversation memory if enabled)
    response = await generate_response_async(message, history)
    
    # Add source citations if enabled
    if SHOW_SOURCES and last_sources_used:
        response += "\n\n---\n**📚 Sources:**\n"
        for i, source in enumerate(last_sources_used, 1):
            response += f"{i}. {source}\n"
    
    # Track empathy (user message + bot response)
    empathy_analyzer.add_user_message(message, response)
    
    # Check if we've reached 10 messages - generate report
    message_count = len(empathy_analyzer.user_messages)
    if message_count == 10:
        report = empathy_analyzer.generate_report()
        if report:
            response += "\n\n" + "="*60 + "\n"
            response += report
            response += "\n" + "="*60
            response += "\n\n💾 **Want to save your data?** Run the export cell below to download as CSV."
    elif message_count < 10:
        # Show progress
        remaining = 10 - message_count
        response += f"\n\n_📊 Empathy tracking: {message_count}/10 messages ({remaining} more for report)_"
    
    return response

# ============================================================================
# STARTER QUESTIONS - OPTIONAL ✏️
# ============================================================================
# These appear as clickable examples when chat starts
# Change these to match your PDFs and persona

STARTER_QUESTIONS = [
    "What are your main beliefs or values?",
    "How did that experience make you feel?",
    "Tell me more about your perspective on this topic.",
    "You seem passionate about this - what drives that feeling?",
    "From your point of view, what are your greatest achievements?",
]

# Create chat interface
memory_status = f"🧠 Conversation memory: {'ENABLED (' + str(CONVERSATION_MEMORY) + ' message pairs)' if CONVERSATION_MEMORY > 0 else 'DISABLED (each question treated independently)'}"

demo = gr.ChatInterface(
    fn=chat_interface,
    title=f"🤖 Chat with {PERSONA_NAME} - Empathy Training",
    description=f"""Practice empathic conversation with {PERSONA_NAME}.
    
    📊 **Empathy Assessment Enabled**
    - Your messages are analyzed for empathy markers
    - Report generated after 10 messages
    - Track: warmth, questions, emotions, perspective, listening
    
    {memory_status}
    {'📖 Source citations enabled - see which PDFs were used' if SHOW_SOURCES else ''}
    
    ⏱️ Response time: 5-15 seconds
    """,
    examples=STARTER_QUESTIONS,
)

# Launch chat
print("=" * 80)
print("🚀 LAUNCHING EMPATHY TRAINING CHAT")
print("=" * 80)
print("\n📊 EMPATHY ASSESSMENT ACTIVE")
print("   • Tracking 5 empathy dimensions")
print("   • Report after 10 messages")
print("   • CSV export available\n")
print(f"\n🧠 CONVERSATION MEMORY: {'ENABLED (' + str(CONVERSATION_MEMORY) + ' exchanges)' if CONVERSATION_MEMORY > 0 else 'DISABLED'}")
if CONVERSATION_MEMORY > 0:
    print(f"   • Bot remembers last {CONVERSATION_MEMORY} message pairs")
    print("   • Follow-up questions work naturally")
    print("   • Change CONVERSATION_MEMORY in Cell 8 to adjust\n")
else:
    print("   • Each question treated independently")
    print("   • Set CONVERSATION_MEMORY > 0 in Cell 8 to enable\n")
print("\n⚠️  IMPORTANT: Use the PUBLIC LINK below (not Colab interface)\n")
if SHOW_SOURCES:
    print("📚 Sources ON - answers show which PDFs were used\n")
print("👇 COPY THIS LINK AND OPEN IN NEW TAB:\n")

demo.launch(
    share=True,      # Create public link
    inline=False,    # Don't show in Colab (unstable)
    debug=True       # Show errors
)

print("\n" + "=" * 80)
print("✅ Chat is live with empathy tracking + conversation memory!")
print("=" * 80)
print("\n📌 STEPS:")
print("   1. Find 'Running on public URL:' above")
print("   2. Copy the https://xxxxx.gradio.live link")
print("   3. Open in new browser tab")
print("   4. Start chatting empathically!")
if CONVERSATION_MEMORY > 0:
    print(f"   5. Try follow-up questions (bot remembers last {CONVERSATION_MEMORY} exchanges)")
    print("   6. After 10 messages, view your empathy report")
else:
    print("   5. After 10 messages, view your empathy report")
if SHOW_SOURCES:
    print(f"   {'7' if CONVERSATION_MEMORY > 0 else '6'}. Check bottom of answers for sources")
print("\n💡 Link expires after 72 hours of no use\n")

---
## 📥 Step 9: Export Conversation Data (Optional)

**Run this after completing your conversation** to download your empathy data as CSV.

This will create a file with:
- All your messages and bot responses
- Empathy scores for each dimension
- Total empathy score per message

You can open this in Excel or Google Sheets for further analysis.

In [ ]:
# Export conversation data to CSV
if len(empathy_analyzer.conversation_history) > 0:
    print("📥 Exporting conversation data...\n")
    
    csv_data = empathy_analyzer.export_to_csv()
    
    # Save to file
    from google.colab import files
    import datetime
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"empathy_conversation_{timestamp}.csv"
    
    with open(filename, 'w') as f:
        f.write(csv_data)
    
    print(f"✅ Data exported to: {filename}")
    print(f"📊 Total messages: {len(empathy_analyzer.user_messages)}")
    
    avg_scores = empathy_analyzer.get_average_scores()
    if avg_scores:
        print(f"📈 Average empathy score: {avg_scores['total']:.1f}/100")
    
    print("\n📥 Downloading file...")
    files.download(filename)
    print("✅ Download complete!")
    print("\n💡 You can now open this CSV file in Excel or Google Sheets")
    
else:
    print("⚠️  No conversation data to export yet!")
    print("💬 Have a conversation first, then run this cell")

---
## 🔄 Step 10: Start New Conversation (Optional)

**Run this to practice empathy again** with a fresh conversation.

This will:
- Reset the empathy tracker (0/10 messages)
- Clear previous conversation history
- Launch a new chat interface

**💡 Tip:** Export your current conversation (Step 9) BEFORE running this!

In [ ]:
# ============================================================================
# RESET & START NEW CONVERSATION
# ============================================================================

print("🔄 Resetting empathy tracker...\n")

# Reinitialize empathy analyzer (clears all previous data)
empathy_analyzer = EmpathyAnalyzer()

print("✅ Empathy tracker reset!")
print("   • Message counter: 0/10")
print("   • Previous conversation cleared")
print(f"   • Memory setting: {CONVERSATION_MEMORY} exchanges")
print("   • Ready for fresh practice\n")

# Relaunch chat interface
print("=" * 80)
print("🚀 LAUNCHING NEW EMPATHY TRAINING CHAT")
print("=" * 80)
print("\n📊 EMPATHY ASSESSMENT ACTIVE")
print("   • Tracking 5 empathy dimensions")
print("   • Report after 10 messages")
print("   • CSV export available\n")
print(f"\n🧠 CONVERSATION MEMORY: {'ENABLED (' + str(CONVERSATION_MEMORY) + ' exchanges)' if CONVERSATION_MEMORY > 0 else 'DISABLED'}")
if CONVERSATION_MEMORY > 0:
    print(f"   • Bot remembers last {CONVERSATION_MEMORY} message pairs")
    print("   • Follow-up questions work naturally")
    print("   • Change CONVERSATION_MEMORY in Cell 8 to adjust\n")
else:
    print("   • Each question treated independently")
    print("   • Set CONVERSATION_MEMORY > 0 in Cell 8 to enable\n")
print("\n⚠️  IMPORTANT: Use the PUBLIC LINK below (not Colab interface)\n")
if SHOW_SOURCES:
    print("📚 Sources ON - answers show which PDFs were used\n")
print("👇 COPY THIS LINK AND OPEN IN NEW TAB:\n")

demo.launch(
    share=True,      # Create public link
    inline=False,    # Don't show in Colab (unstable)
    debug=True       # Show errors
)

print("\n" + "=" * 80)
print("✅ New conversation started!")
print("=" * 80)
print("\n📌 STEPS:")
print("   1. Find 'Running on public URL:' above")
print("   2. Copy the https://xxxxx.gradio.live link")
print("   3. Open in new browser tab")
print("   4. Start your new empathy practice!")
if CONVERSATION_MEMORY > 0:
    print(f"   5. Try follow-up questions (bot remembers last {CONVERSATION_MEMORY} exchanges)")
print("\n💡 Remember: Export your previous conversation first if you haven't already\n")

---
## 🔧 Troubleshooting

### API Token Issues:
- **Error: "Invalid token"**
  - Get a new token from: https://huggingface.co/settings/tokens
  - Make sure you copied the entire token (starts with `hf_`)
  - Replace `YOUR_API_TOKEN_HERE` in Step 4

### Rate Limit Issues:
- **Error: "Rate limit reached"**
  - Free tier: ~300 requests/hour
  - Wait 10-15 minutes before trying again
  - Consider upgrading to PRO ($9/month) for 20x more requests

### PDF Issues:
- **"File not found" errors:**
  - Check that Google Drive is mounted (Step 3)
  - Verify PDF file paths are correct
  - Make sure paths start with `/content/drive/MyDrive/`
  
- **"No text extracted":**
  - PDF might be scanned images (not searchable text)
  - PDF might be password-protected
  - Try opening the PDF to verify it has selectable text

### Response Issues:
- **Responses don't match persona:**
  - Make `PERSONA_DESCRIPTION` more detailed and specific
  - Add more example phrases/words they use
  
- **Responses aren't relevant:**
  - Increase `NUM_RETRIEVED_DOCS` (try 5 or 7)
  - Make sure PDFs contain information about the topic
  - Ask more specific questions

### Model Issues:
- **Slow responses:**
  - Try a smaller model (microsoft/Phi-3-mini-4k-instruct)
  - Reduce MAX_OUTPUT_TOKENS
  
- **Model not available:**
  - Check model name at https://huggingface.co/models
  - Try alternative models listed in Cell 8

### Performance Issues:
- **Colab disconnects or times out:**
  - This is normal for free Colab after ~12 hours
  - Save your work and restart
  - Keep the browser tab active

### Need More Help?
- Check HuggingFace status: https://status.huggingface.co/
- HuggingFace documentation: https://huggingface.co/docs/api-inference/
- Verify free tier limits at: https://huggingface.co/pricing